In [3]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder.getOrCreate()

In [6]:
# 1. Crear un DataFrame con datos de personas, que incluya nombre, edad y ciudad de residencia.
data = [("Alice", 25, "New York"),
        ("Bob", 30, "Los Angeles"),
        ("Charlie", 22, "Chicago")]

df = spark.createDataFrame(data, ["nombre", "edad", "ciudad"])
df.show()

+-------+----+-----------+
| nombre|edad|     ciudad|
+-------+----+-----------+
|  Alice|  25|   New York|
|    Bob|  30|Los Angeles|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [18]:
# 2. Mostrar solo los nombres de las personas del DataFrame.
df.select("nombre").show()

+-------+
| nombre|
+-------+
|  Alice|
|    Bob|
|Charlie|
+-------+



In [19]:
# 3. Filtrar personas cuya edad sea mayor o igual a 25.
df.filter(df.edad >= 25).show()

+------+----+-----------+
|nombre|edad|     ciudad|
+------+----+-----------+
| Alice|  25|   New York|
|   Bob|  30|Los Angeles|
+------+----+-----------+



In [25]:
# 4. Agregar una nueva columna "Pais" con un valor constante para todas las filas.
from pyspark.sql.functions import lit
df.withColumn("Pais", lit("Peru")).show()

+-------+----+-----------+----+
| nombre|edad|     ciudad|Pais|
+-------+----+-----------+----+
|  Alice|  25|   New York|Peru|
|    Bob|  30|Los Angeles|Peru|
|Charlie|  22|    Chicago|Peru|
+-------+----+-----------+----+



In [7]:
# 5. Calcular el promedio de edad de todas las personas.
df.agg( avg("edad").alias("Promedio Edad")
        ).show()

+------------------+
|     Promedio Edad|
+------------------+
|25.666666666666668|
+------------------+



In [28]:
# 6. Ordenar el DataFrame por edad en orden descendente.
df.sort(desc("edad")).show()

+-------+----+-----------+
| nombre|edad|     ciudad|
+-------+----+-----------+
|    Bob|  30|Los Angeles|
|  Alice|  25|   New York|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [29]:
# 7. Agrupar por ciudad y calcular la cantidad de personas en cada ciudad.
(df.groupBy("ciudad")
    .agg(
        count("nombre").alias("count")
        ).show()
)

+-----------+-----+
|     ciudad|count|
+-----------+-----+
|   New York|    1|
|Los Angeles|    1|
|    Chicago|    1|
+-----------+-----+



In [30]:
# 8. Renombrar la columna "Nombre" a "NombreCompleto".
df = df.withColumnRenamed('nombre', 'NombreCompleto')
df.show()

+--------------+----+-----------+
|NombreCompleto|edad|     ciudad|
+--------------+----+-----------+
|         Alice|  25|   New York|
|           Bob|  30|Los Angeles|
|       Charlie|  22|    Chicago|
+--------------+----+-----------+



In [32]:
# 9. Eliminar la columna "Edad" del DataFrame.
newdf=df
newdf = newdf.drop("edad")
newdf.show()

+--------------+-----------+
|NombreCompleto|     ciudad|
+--------------+-----------+
|         Alice|   New York|
|           Bob|Los Angeles|
|       Charlie|    Chicago|
+--------------+-----------+



In [53]:
# 10. Realizar una consulta SQL en el DataFrame para seleccionar personas mayores de 20 años.
df.createOrReplaceTempView("temp_table")
spark.sql("select * from temp_table where edad > 20").show()

+-------+----+-----------+
| nombre|edad|     ciudad|
+-------+----+-----------+
|  Alice|  25|   New York|
|    Bob|  30|Los Angeles|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [35]:
# 11. Calcular la suma total de todas las edades.
spark.sql("select sum(edad) from temp_table").show()

+---------+
|sum(edad)|
+---------+
|       77|
+---------+



In [36]:
# 12. Calcular la edad mínima y máxima de todas las personas.
spark.sql("select min(edad), max(edad) from temp_table").show()

+---------+---------+
|min(edad)|max(edad)|
+---------+---------+
|       22|       30|
+---------+---------+



In [41]:
# 13. Filtrar personas cuya ciudad de residencia sea "Chicago" y edad sea menor de 30.
spark.sql("select * from temp_table where ciudad = 'Chicago' and edad < 30").show()

+--------------+----+-------+
|NombreCompleto|edad| ciudad|
+--------------+----+-------+
|       Charlie|  22|Chicago|
+--------------+----+-------+



In [42]:
# 14. Agregar una nueva columna "EdadDuplicada" que contenga el doble de la edad.
df.withColumn("EdadDuplicada", col("edad") * 2).show()

+--------------+----+-----------+-------------+
|NombreCompleto|edad|     ciudad|EdadDuplicada|
+--------------+----+-----------+-------------+
|         Alice|  25|   New York|           50|
|           Bob|  30|Los Angeles|           60|
|       Charlie|  22|    Chicago|           44|
+--------------+----+-----------+-------------+



In [23]:
# 15. Convertir todas las edades en años a meses.
df.withColumn("EdadMeses", col("edad") * 12).show()

+-------+----+-----------+---------+
| nombre|edad|     ciudad|EdadMeses|
+-------+----+-----------+---------+
|  Alice|  25|   New York|      300|
|    Bob|  30|Los Angeles|      360|
|Charlie|  22|    Chicago|      264|
+-------+----+-----------+---------+



In [45]:
# 16. Contar el número total de personas en el DataFrame.
spark.sql("select count(NombreCompleto) from temp_table").show()

+---------------------+
|count(NombreCompleto)|
+---------------------+
|                    3|
+---------------------+



In [11]:
# 17. Filtrar personas cuya edad sea un número par.
from pyspark.sql.functions import filter
df.filter(df.edad % 2 == 0).show()

+-------+----+-----------+
| nombre|edad|     ciudad|
+-------+----+-----------+
|    Bob|  30|Los Angeles|
|Charlie|  22|    Chicago|
+-------+----+-----------+



In [ ]:
# 18. Calcular la cantidad de personas por rango de edades (0-20, 21-40, 41-60, 61+).


In [50]:
# 19. Contar cuántas personas tienen el mismo nombre.
spark.sql("select NombreCompleto, count(0) from temp_table group by NombreCompleto").show()

+--------------+--------+
|NombreCompleto|count(0)|
+--------------+--------+
|         Alice|       1|
|           Bob|       1|
|       Charlie|       1|
+--------------+--------+



In [21]:
# 20. Concatenar las columnas "Nombre" y "Ciudad" en una nueva columna llamada "InformacionPersonal".
from pyspark.sql.functions import concat_ws,col
df.select("nombre","edad","ciudad",concat_ws(' ',df.nombre,df.ciudad).alias("InformacionPersonal")).show()

+-------+----+-----------+-------------------+
| nombre|edad|     ciudad|InformacionPersonal|
+-------+----+-----------+-------------------+
|  Alice|  25|   New York|     Alice New York|
|    Bob|  30|Los Angeles|    Bob Los Angeles|
|Charlie|  22|    Chicago|    Charlie Chicago|
+-------+----+-----------+-------------------+

